In [1]:
# データの用意
import glob
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
x_train = np.empty([0, 224, 224, 3])
y_train = np.empty([0])
IMG_SIZE = 224
for i in range(16):
    files = glob.glob(f'datas/major_org/{i}/*.jpg')
    imgs = np.array([img_to_array(load_img(f, color_mode='rgb', target_size=(IMG_SIZE,IMG_SIZE))) for f in files])
    labels = np.array([i for f in files])

    x_train = np.r_[x_train, imgs]
    y_train = np.r_[y_train, labels]

files = glob.glob(f'datas/tests/*.jpg')
x_test = np.array([img_to_array(load_img(f, color_mode='rgb', target_size=(IMG_SIZE,IMG_SIZE))) for f in files])
y_test = np.array([int(f.replace('datas/tests\\', '').replace('.jpg', '')) for f in files])

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_test = np.pad(y_test, [(0,0),(0,1)], 'constant')

In [2]:
# データセットの大きさ確認
print(x_train.shape) # 訓練データのサイズ
print(y_train.shape) 
print(x_test.shape) # テストデータのサイズ
print(y_test.shape)
#(464, 224, 224, 3)

(464, 224, 224, 3)
(464, 16)
(15, 224, 224, 3)
(15, 16)


In [ ]:
# データセットの表示
%matplotlib inline
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(120, 120))
fig.subplots_adjust(left=0, right=1, bottom=0, top=0.5, hspace=0.05, wspace=0.05)

for i in range(9):
    ax = fig.add_subplot(1, 9, i + 1, xticks=[], yticks=[])
    ax.imshow((x_train[i] * 255).astype(np.uint8))

In [ ]:
# モデル構築 ふんわりCNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Convolution2D, MaxPooling2D, ZeroPadding2D

model = Sequential()

model.add(Convolution2D(32, (3, 3), activation='relu', padding='same', input_shape=(120, 120, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Convolution2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(256, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(364, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))

# 出力層
model.add(Dense(16, activation='softmax'))

model.compile(loss="categorical_crossentropy", metrics = ["accuracy"], optimizer = "adam")

In [2]:
# モデル構築 DenseNet
from tensorflow.keras.applications import densenet
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
base_model = densenet.DenseNet121(include_top=False, weights='imagenet', classes=1000)

x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(16, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(loss="categorical_crossentropy", metrics = ["accuracy"], optimizer = "adam")

In [3]:
# 学習
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import image

# data augmentationの設定
params = {
    'rotation_range': 20,
    'width_shift_range':0.3,
    'height_shift_range':0.3
}
datagen = image.ImageDataGenerator(**params)
datagen.fit(x_train)

early_stopping = EarlyStopping(monitor='loss', patience=5, verbose=1)

history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=1024), epochs=100, callbacks=[early_stopping])
# model.fit(x_train, y_train, batch_size=100, verbose=2, epochs=30, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/100


In [ ]:
# 性能評価
score = model.evaluate(x_test, y_test, verbose=0)
print(score)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# 経過確認
import matplotlib.pyplot as plt
print(history.history.keys())
acc = history.history['accuracy']
epochs = range(1, len(acc)+1)
plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.legend()
plt.show()

In [ ]:
loss = history.history['loss']
epochs = range(1, len(loss)+1 )
plt.plot(epochs, loss, 'b', label='Training loss')
plt.legend()
plt.show()